# Imports

In [44]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from PIL import Image

## Define image size and age bins

In [45]:
IMAGE_SIZE = (128, 128)
AGE_BINS = [0, 12, 20, 30, 60, 100]
AGE_LABELS = ['Child', 'Teen', 'Young Adult', 'Adult', 'Elderly']
GENDER_LABELS = ['Male', 'Female']

## Data Loading and Preprocessing


In [46]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
# Step 2: Define paths
import os
import zipfile

zip_path_expr = '/content/drive/MyDrive/Face expression.zip'
expr_dir = '/content/face-expression-recognition-dataset'

zip_path_utk = '/content/drive/MyDrive/UTKFace.zip'
utkface_dir = '/content/UTKFace'

# Extract the Face Expression dataset
with zipfile.ZipFile(zip_path_expr, 'r') as zip_ref:
    zip_ref.extractall(expr_dir)
print("Face Expression dataset extracted successfully.")

# Extract the UTKFace dataset
with zipfile.ZipFile(zip_path_utk, 'r') as zip_ref:
    zip_ref.extractall(utkface_dir)
print("UTKFace dataset extracted successfully.")

Face Expression dataset extracted successfully.
UTKFace dataset extracted successfully.


In [48]:
expr_train_path = '/content/face-expression-recognition-dataset/images/train'
expr_val_path = '/content/face-expression-recognition-dataset/images/validation'
utk_path = '/content/UTKFace/utkface_aligned_cropped/crop_part1'

## create dataframe

In [49]:
def parse_utkface_metadata(folder_path):
    ages, genders, image_paths = [], [], []
    for fname in os.listdir(folder_path):
        try:
            split = fname.split('_')
            ages.append(int(split[0]))
            genders.append(int(split[1]))
            image_paths.append(os.path.join(folder_path, fname))
        except:
            continue
    df = pd.DataFrame({
        'path': image_paths,
        'age': ages,
        'gender': genders
    })
    df['age_group'] = pd.cut(df['age'], bins=AGE_BINS, labels=AGE_LABELS, right=False)
    df['gender_label'] = df['gender'].map({0: 'Male', 1: 'Female'})
    return df

## Load FER expression data and create dataframe

In [50]:
def load_expression_data(base_dir):
    data = []
    label_map = {}
    for i, emotion in enumerate(sorted(os.listdir(base_dir))):
        emotion_dir = os.path.join(base_dir, emotion)
        if not os.path.isdir(emotion_dir):
            continue
        label_map[emotion] = i
        for fname in os.listdir(emotion_dir):
            path = os.path.join(emotion_dir, fname)
            if os.path.isfile(path):
                data.append({'path': path, 'expression': emotion})
    df = pd.DataFrame(data)
    return df, label_map


## Load and preprocess images

In [51]:
def load_images_from_df(df, label_column, image_size):
    images, labels = [], []
    for _, row in df.iterrows():
        img_path = row['path']
        if not os.path.isfile(img_path):
            print(f"Skipped (not a file): {img_path}")
            continue
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipped (not an image or corrupted): {img_path}")
            continue
        img = cv2.resize(img, image_size)
        images.append(img)
        labels.append(row[label_column])
    return np.array(images), labels


## Create metadata and load images

In [52]:
utk_df = parse_utkface_metadata(utk_path)
expr_train_df, expr_map = load_expression_data(expr_train_path)
expr_val_df, _ = load_expression_data(expr_val_path)

In [53]:
utk_imgs, utk_age_labels = load_images_from_df(utk_df, 'age_group', IMAGE_SIZE)
_, utk_gender_labels = load_images_from_df(utk_df, 'gender_label', IMAGE_SIZE)

expr_train_imgs, expr_train_labels = load_images_from_df(expr_train_df, 'expression', IMAGE_SIZE)
expr_val_imgs, expr_val_labels = load_images_from_df(expr_val_df, 'expression', IMAGE_SIZE)

## Preprocess and encode labels

In [ ]:
le_age = LabelEncoder()
le_gender = LabelEncoder()
le_expr = LabelEncoder()

age_encoded = to_categorical(le_age.fit_transform(utk_age_labels))
gender_encoded = to_categorical(le_gender.fit_transform(utk_gender_labels))
expr_train_encoded = to_categorical(le_expr.fit_transform(expr_train_labels))
expr_val_encoded = to_categorical(le_expr.transform(expr_val_labels))

# Normalize pixel values
utk_imgs = utk_imgs.astype('float32') / 255.0
expr_train_imgs = expr_train_imgs.astype('float32') / 255.0
expr_val_imgs = expr_val_imgs.astype('float32') / 255.0

## Split datasets

In [ ]:
X_age_train, X_age_test, y_age_train, y_age_test = train_test_split(
    utk_imgs, age_encoded, test_size=0.2, random_state=42
)

X_gender_train, X_gender_test, y_gender_train, y_gender_test = train_test_split(
    utk_imgs, gender_encoded, test_size=0.2, random_state=42
)

X_expr_train, X_expr_test, y_expr_train, y_expr_test = train_test_split(
    expr_train_imgs, expr_train_encoded, test_size=0.2, random_state=42
)

## Build Multi-Output CNN Model

In [ ]:
input_tensor = Input(shape=(128, 128, 3))
base_model = MobileNetV2(input_tensor=input_tensor, include_top=False, weights='imagenet')
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)

# Age prediction head
age_head = Dense(64, activation='relu')(x)
age_output = Dense(len(AGE_LABELS), activation='softmax', name='age')(age_head)

# Gender prediction head
gender_head = Dense(64, activation='relu')(x)
gender_output = Dense(len(GENDER_LABELS), activation='softmax', name='gender')(gender_head)

# Expression prediction head
expr_head = Dense(64, activation='relu')(x)
expr_output = Dense(len(expr_map), activation='softmax', name='expression')(expr_head)

# Combine into model
multi_model = Model(inputs=input_tensor, outputs=[age_output, gender_output, expr_output])

multi_model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        'age': 'categorical_crossentropy',
        'gender': 'categorical_crossentropy',
        'expression': 'categorical_crossentropy'
    },
    metrics=['accuracy']
)

multi_model.summary()

## Train model separately on tasks

In [ ]:
multi_model.fit(
    X_age_train,
    {'age': y_age_train, 'gender': np.zeros_like(y_gender_train), 'expression': np.zeros_like(y_expr_train)},
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    verbose=2
)

# Train on Gender only
multi_model.fit(
    X_gender_train,
    {'age': np.zeros_like(y_age_train), 'gender': y_gender_train, 'expression': np.zeros_like(y_expr_train)},
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    verbose=2
)

# Train on Expression only
multi_model.fit(
    X_expr_train,
    {'age': np.zeros_like(y_age_train), 'gender': np.zeros_like(y_gender_train), 'expression': y_expr_train},
    epochs=5,
    batch_size=32,
    validation_split=0.1,
    verbose=2
)

## Save and convert to TFLite

In [ ]:
multi_model.save("age_gender_expression_model.h5")
print("Keras model saved as 'age_gender_expression_model.h5'")

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(multi_model)
tflite_model = converter.convert()

with open("age_gender_expression_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model exported as 'age_gender_expression_model.tflite'")